## Comparing shared ORFs to ANI per genome

---  
#### Calculating the number of orthologous genes per genome pair at different pct_id thresholds, and merging this information with calculated whole genome ANI values per pair.

#### High level steps:
1. Run pyANI comparison of all genomes
1. Call genes on genomes using prokka (I used standard SCGC prokka outputs in the original analysis)
    a. make sure that gene names follow a pattern in which the first identifier per gene name identifies the genome the gene came from, followed by an underscore (e.g. AG-910-A01_gene1)  
    b. copy appropriately formatted \*.fnn files to a working data directory (e.g. data/fnn)
2. Create a BLAST database of all ORFs in genome collection
3. Run BLASTn of each individual ffn against the database of all ORFs
4. Summarize each BLAST file per genome pair, write each summary per SAG to a file
5. Combine all summaries into one file, combine with pyANI output


--- 
### Specific instructions

#### Compare Genomes via pyANI

e.g.  

```
module load anaconda3/2019.07

indir=data/contigs/
outdir=analyses/pyani/

average_nucleotide_identity.py -o $outdir \
-i ${indir} -m ANIb --workers 20
```

#### BLASTn Open Reading Frame Comparison
See the following cells for code for the BLASTn analysis

1. Begin with called ORF na sequences from each genome, placed in the same directory.  
    a. I copied all \*.fnn files from each SAG in the collection into the directory data/ffn/
2. Create BLAST database of all ORFs from all genomes that you want to compare with below code:

In [ ]:
import glob
import os.path as op
import shutil
import itertools

def readfa(fh):
    for header, group in itertools.groupby(fh, lambda line: line[0] == '>'):
        if header:
            line = next(group)
            name = line[1:].strip()
        else:
            seq = ''.join(line.strip() for line in group)
            yield name, seq
            
def write_fa_record(name, seq, oh, line_len=60):
    print(">{}".format(name), file=oh)
    for i in range(0, len(seq), line_len):
        print(seq[i:i+line_len], file=oh)

def get_sag_orfs(sag, prokka_dir):
    ffn = op.join(prokka_dir, "{}.ffn".format(sag, sag))
    if op.exists(ffn):
        return ffn
    else:
        print("could not find ffn for {}".format(sag))
        return None

def safe_makedir(dname):
    """
    Make a directory if it doesn't exist, handling concurrent race conditions.
    """
    if not dname:
        return dname
    num_tries = 0
    max_tries = 5
    while not os.path.exists(dname):
        try:
            os.makedirs(dname)
        except OSError:
            if num_tries > max_tries:
                raise
            num_tries += 1
            time.sleep(2)
    return dname

define where you want to write the ffn and blast database:

In [ ]:
ffn_dir = safe_makedir("data/ffn/")
database = safe_makedir("data/blast_database/")
db_fasta = op.join(database, "combined_ffas.fasta")

In [ ]:
ffns = glob.glob(ffn_dir, "*.ffn")

with open(db_fasta, 'w') as oh:
    for ffn in ffns:
        for name, seq in readfa(open(ffn)):
            write_fa_record(name, seq, oh)

In [ ]:
!makeblastdb -in {db_fasta} -dbtype nucl

#### BLAST each genome individually against the database of all genomes.  
    a. see blastn.snakemake, pasted below, edit as appropriate  
    b. pay attention to the max-target-seqs parameter of blast... this may be important if you have a lot of similar genomes in your collection (as we did in GORG-Tropics)...  
    
blastn.snakemake:

```
import os.path as op
import os

### EDIT ME ###

# path to directory with all ffn files:
ffn_dir = "/mnt/scgc/scgc_nfs/lab/julia/notebooks/panspecies/fig3/data/ffn/"

# path to BLAST database
database = "/mnt/scgc/scgc_nfs/lab/julia/notebooks/panspecies/fig3/data/blast_database/gorg-tropics_16s_80pct_complete_orfs.fasta"

# path to directory to write all BLAST outputs to
outdir = "/mnt/scgc/scgc_nfs/lab/julia/notebooks/panspecies/fig3/analyses/blastn/"

SAMPLES, = glob_wildcards(op.join(ffn_dir,'{sample}.ffn'))

# path to write snakemake logs to
logdir = '/home/julia/out/210818_snakemake'

### 

rule all:
    input:
        expand(op.join(outdir, '{sample}_vs_gt-16s-80comp-orfs.out'), sample=SAMPLES)

rule blastn:
    input:
        ffn = op.join(ffn_dir, '{sample}.ffn')
    output:
        outblast = op.join(outdir, '{sample}_vs_gt-16s-80comp-orfs.out')
    params:
        db = database,
        threads = 8,
        max_seqs = 2000
    resources:
        queue = 'route',
        cpus = '10',
        walltime = '5:00:00',
        mem = '10G',
        log = logdir
    shell:
        "module purge; module load anaconda3; \
        blastn -db {params.db} \
        -query {input.ffn} \
        -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen' \
        -num_threads {params.threads} -max_target_seqs {params.max_seqs} \
        -out {output.outblast}"
        

# How to submit to scheduler using this snakemake file:
'''snakemake -s /mnt/scgc/scgc_nfs/lab/julia/notebooks/panspecies/fig3/subs/blastn.snakemake --cluster \
"qsub -q {resources.queue} -j oe -o {resources.log} -l ncpus={resources.cpus},mem={resources.mem},walltime={resources.walltime}" \
-j 900 -k --latency-wait 20'''
```

Process and summarize BLAST outputs

In [2]:
import pandas as pd
import os
import os.path as op
import glob
import glob
from collections import Counter
import numpy as np
import itertools

# You will need a table summarizing gene counts per SAG

def create_orf_summary_table(ffn_list, outfile, length_threshold = [500, 1500]):
    with open(outfile, 'w') as oh:
        print('sag','gene_count','gene_count_{}-{}bp'.format(len_threshold[0], len_threshold[1]), sep = ',', file = oh)
        for ffn in ffn_list:
            sag = op.basename(ffn).split(".")[0]
            gene_count = 0
            thresh_count = 0
            for name, seq in readfa(open(ffn)):
                gene_count += 1
                gene_length = len(seq)
                if len_threshold[0] <= gene_length and len_threshold[1] >= gene_length:
                    thresh_count += 1
            print(sag, gene_count, thresh_count, sep = ',', file = oh)

In [ ]:
path_to_blast_outputs = "analyses/blastn/"
ffn_dir = "data/ffn/"

# new files and directories
table_dir = safe_makedir("tbls")
orf_summary_table = op.join(table_dir, "sag_orf_summary.csv")

# list of all blast outputs
blasts = glob.glob(op.join(path_to_blast_outputs, "*.out"))

# list of all ffns
ffns = glob.glob(op.join(ffn_dir, "*.ffn"))

create_orf_summary_table(ffns, orf_summary_table)

### Process each of the blast files, write each to a new summary file

In [ ]:
# function to process blast files:

def process_blast(blastin, 
                  orf_count_df,
                  pid_list = [99.9, 99.8, 99.7, 99.6, 99.5, 99, 98.5, 98, 97.5, 97], 
                  len_threshold = [500, 1500]):
    names = 'qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen'.split()
    df = pd.read_csv(blastin, sep = "\t", names = names)

    df['qsag'] = [i.split("_")[0] for i in df['qseqid']]              
    df['ssag'] = [i.split("_")[0] for i in df['sseqid']]
    df = df[df['qsag'] != df['ssag']]
    df = df[(df['length'] > (0.8 * df['qlen'])) | (df['length'] > 0.8 * df['slen'])]
    
    newdf = pd.DataFrame(columns = ['qsag','ssag'])
    
    ni_alpha_mean = df.groupby(['qsag','ssag'], as_index = False)['pident'].mean().rename(columns = {'pident':'mean_pident'}).merge(
                    df.groupby(['qsag','ssag'])['pident'].std().reset_index().rename(columns = {'pident':'stdev_pident'})).merge(
                    df.groupby(['qsag','ssag'], as_index = False)['pident'].count().rename(columns = {'pident':'total_hits'})).merge(
                    df.groupby(['qsag','ssag'], as_index = False)['pident'].median().rename(columns = {'pident':'median_pident'}))
    
    newdf = newdf.merge(ni_alpha_mean, how = 'outer').merge(
            orf_count_df.rename(columns = {i:'s{}'.format(i) for i in orf_count_df.columns}), how = 'left').merge(
            orf_count_df.rename(columns = {i:'q{}'.format(i) for i in orf_count_df.columns}), how = 'left')
    
    for pid in pid_list:
        
        len_bound = df[(df['pident'] >= pid) & 
                       ((df['qlen'] >= len_threshold[0]) & (df['qlen'] <= len_threshold[1])) & 
                      ((df['slen'] >= len_threshold[0]) & (df['slen'] <= len_threshold[1]))]\
                        .groupby(['qsag','ssag'], as_index=False)['sseqid']\
                        .count().rename(columns = {'sseqid':'{}_pid_{}-{}bp_orthologs'.format(pid, len_threshold[0],len_threshold[1])})
        all_sizes = df[df['pident'] >= pid].groupby(['qsag','ssag'], as_index=False)['sseqid']\
                        .count().rename(columns = {'sseqid':'{}_pid_orthologs'.format(pid, len_threshold[0],len_threshold[1])})
        newdf = newdf.merge(
                len_bound, how = 'outer').merge(
                all_sizes, how = 'outer')
        
    return newdf

Apply function to all blast files, write each to an summary file per blast file

In [ ]:
blast_summary_dir = safe_makedir("analyses/blastn_summaries")

orf_count_df = pd.read_csv(orf_summary_table)

blasts = glob.glob(op.join(path_to_blast_outputs,"*")

for blast in blasts:
    out = op.join(blast_summary_dir, '{}_summary.csv'.format(op.basename(blast).split("_")[0]))
    process_blast(blast, orf_count_df).sort_values(by = 'total_hits', ascending=False).to_csv(out, index=False)

In [ ]:
# load all results into a large dataframe:

giantdf = pd.concat([pd.read_csv(i) for i in glob.glob('analyses/blast_summaries/*')])

# add a pairs column to have a unique identifier per genome pair:
pairs = []

for i, l in giantdf.iterrows():

    lst = [l['qsag'], l['ssag']]
    lst.sort()
    pairs.append("_".join(lst))

giantdf['pair'] = pairs

ortho_names = [i for i in giantdf.columns if 'orthologs' in i]

# dropping redundant counts, as this was essentially a reciprocal BLAST... have confirmed this in other analyses
sdf = giantdf.drop_duplicates(subset = 'pair')

#### Load pyani results

In [ ]:
def load_pyani_results(pyani_outdir, method='ANIb', min_coverage = 0.03, scgc=True):
    identity = op.join(pyani_outdir,'{}_percentage_identity.tab'.format(method))
    lengths = op.join(pyani_outdir, '{}_alignment_lengths.tab'.format(method))
    coverage = op.join(pyani_outdir, '{}_alignment_coverage.tab'.format(method))
    simerrors = op.join(pyani_outdir, '{}_similarity_errors.tab'.format(method))
    
    load_matrix_table = lambda metric, table: pd.read_csv(table, sep="\t").melt(id_vars='Unnamed: 0', var_name='Genome B', value_name=metric).rename(columns={'Unnamed: 0':'Genome A'})
    
    idf = load_matrix_table('ani', identity)
    ldf = load_matrix_table('ani_aln_len', lengths)
    cdf = load_matrix_table('ani_aln_cov', coverage)
    edf = load_matrix_table('ani_similarity_errs', simerrors)
    
    bdf = idf.merge(ldf).merge(cdf).merge(edf)
    bdf = bdf[bdf['Genome A'] != bdf['Genome B']]
    
    if scgc:
        bdf['Genome A'] = [i.split("_")[0] for i in bdf['Genome A']]
        bdf['Genome B'] = [i.split("_")[0] for i in bdf['Genome B']]
    
    comps = []

    for i, l in bdf.iterrows():
        lst = [l['Genome A'], l['Genome B']]
        lst.sort()
        string = "{}_{}".format(lst[0], lst[1])
        comps.append(string)
    bdf['comp'] = comps
    bdf = bdf.sort_values(by=['comp','ani'], ascending=False)
    
    dfa = bdf.drop_duplicates(subset=['comp'], keep='first')
    dfb = bdf.drop_duplicates(subset=['comp'], keep='last')
    dfa.rename(columns={'ani_aln_cov':'ani_aln_cov_ab', 'ani_aln_len':'ani_aln_len_ab', 
                       'ani_similarity_errs':'ani_similarity_errs_ab'}, inplace=True)
    dfb.rename(columns={'ani_aln_cov':'ani_aln_cov_ba', 'ani_aln_len':'ani_aln_len_ba', 
                       'ani_similarity_errs':'ani_similarity_errs_ba'}, inplace=True)

    dfab = dfa.merge(dfb[['comp', 'ani_aln_cov_ba','ani_aln_len_ba','ani_similarity_errs_ba']], on='comp', how='outer')
    outdf = dfab[(dfab['ani_aln_cov_ba'] > min_coverage) & (dfab['ani_aln_cov_ab'] > min_coverage)]
    return outdf

# load the ani df created by pyani:

pyanidir = "analyses/pyaniout/"

anidf = load_pyani_results()
pairs2  = []

for i, l in anidf.iterrows():
    lst = [l['Genome A'], l['Genome B']]
    lst.sort()
    pairs2.append("_".join(lst))

anidf = anidf.rename(columns = {'comp':'pair'})

anidf = anidf[['Genome A','Genome B','ani', 'ani_aln_coverage_ab','ani_aln_coverage_ba', 'pair']].dropna()

Merge anidf and the dereplicated BLAST ortholog comparison table

In [ ]:
sdf = sdf.merge(anidf, on = 'pair', how = 'outer')

# add GND for Ramunas
sdf['GND'] = [round(100 - i * 100, 2) for i in sdf['ani']]

# include all pairs for which ANI was measured in summary calculations:
sdf[ortho_names] = sdf[ortho_names].fillna(0)

# create ani bins
tups = [(round(i-0.01, 2), round(i, 2),) for i in np.arange(1, 0.6, -0.01)]
bins = pd.IntervalIndex.from_tuples(tups)
sdf['ani_bin'] = pd.cut(sdf['ani'], bins)

# write to csv:
sdf.to_csv("tbls/sag_pair_summary_shared_orfs_ani.csv", index = False)